In [1]:
from lib.sentence2vec import Sentence2Vec
import pandas as pd

F:\software\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
data = pd.read_csv('../feature/data_cutted.csv')
model = Sentence2Vec('../feature/content_100.model')

sen2vec = pd.DataFrame(data['content'].apply(lambda x:model.get_vector(str(x))).tolist())

data_df = pd.concat([data,sen2vec],axis=1)
data_df.drop(['content'],axis=1,inplace=True)

#data_df.to_csv('../feature/data_vec.csv',index=False)
#data_df = pd.read_csv('../feature/data_vec.csv')

train = data_df.loc[data_df['source']=='train']
test = data_df.loc[data_df['source']=='test']

train.drop(['source'],axis=1,inplace=True)
test.drop(['source'],axis=1,inplace=True)

del data_df

C:\Users\kuhung\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\kuhung\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [3]:
train.head()

,id,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,yhM4ivIm,-0.011067,-0.002076,0.004751,0.008949,0.004679,0.013545,0.006279,0.021481,0.011637,...,0.010411,0.012543,-0.011424,0.008330,-0.002789,0.010611,-0.005866,0.014511,0.026303,-0.005738
1,OeJidjGX,-0.018658,0.032255,-0.027114,0.017345,0.010428,0.059051,0.014392,0.038320,0.038810,...,0.005236,0.021921,-0.061973,0.009029,-0.021268,0.032886,0.006841,0.017118,0.084283,-0.020280
2,4wWp3dGK,-0.000766,-0.004048,0.023685,-0.002038,0.011975,0.020104,-0.031096,0.006151,-0.009893,...,-0.018228,-0.008004,-0.034211,-0.019323,-0.006644,-0.003683,0.005089,0.001706,-0.006472,-0.026680
3,T0senQto,-0.054802,0.011654,-0.052005,0.046572,-0.004628,0.017028,-0.011227,0.059478,0.035232,...,-0.021379,-0.060894,-0.064221,0.023103,-0.044009,-0.018016,0.015297,-0.035342,0.094760,0.016432
4,1jAboq7C,-0.001775,-0.025761,0.006165,0.014823,-0.000683,0.000681,-0.013310,0.005354,0.013457,...,0.005769,-0.008306,-0.023156,0.000741,-0.011380,-0.008875,0.006091,0.014180,0.005470,-0.013609


In [4]:
X = train.drop(['id'],axis=1)

In [5]:
y = train['id']

In [6]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(X, y) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [53]:
feature = test.drop(['id'],axis=1)
result = neigh.predict(feature)

sub = pd.DataFrame({'0':test['id'],'1':result})
sub.columns=['test_id','result']

In [54]:
# compute similar
sub.head()

,test_id,result
99544,ahpq2Zzn,HeGxWnKV
99545,p2Hkot4E,EemG3TYB
99546,6pEJaYSB,nboyHcTL
99547,fEClIHUb,DtWPQnv4
99548,QORyZCX6,viqACuS2


In [55]:
def get_distance(x,y):
    return model.similarity(data[data.id==x]['content'].to_string(),data[data.id==y]['content'].to_string())

sub['distance']=sub.apply(lambda row: get_distance(row['test_id'], row['result']), axis=1)

F:\kuhung\Desktop\LAB\SMP ETST 2018\sentence2vec-master\lib\sentence2vec.py:40: RuntimeWarning: invalid value encountered in double_scalars
  score = dot(xv, yv) / (norm(xv) * norm(yv))


In [56]:
sub.head()

,test_id,result,distance
99544,ahpq2Zzn,HeGxWnKV,0.926980
99545,p2Hkot4E,EemG3TYB,0.690158
99546,6pEJaYSB,nboyHcTL,0.892106
99547,fEClIHUb,DtWPQnv4,0.681556
99548,QORyZCX6,viqACuS2,0.766129


In [57]:
sub.describe()

,distance
count,998.000000
mean,0.858690
std,0.110571
min,0.350115
25%,0.796180
50%,0.876988
75%,0.945252
max,1.000000


In [58]:
def threshold(result,distance):
    if distance > 0.876988:
        return result
    else:
        return 

In [59]:
sub['result'] = sub.apply(lambda row: threshold(row['result'],row['distance']),axis=1)

In [60]:
sub.drop('distance',inplace=True,axis=1)

In [61]:
sub.replace({'﻿mhY5opF':'?mhY5opF4'},inplace=True)

In [62]:
sub.to_csv('../output/sen2vec_threshold50.csv',index=False)